In [ ]:
# Explore Angles - Visualize episode with angle cones
import numpy as np
import zarr
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge, Circle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

# Load dataset
DATASET_PATH = '../../dataset/margit_100_256x144.zarr'
root = zarr.open(DATASET_PATH, mode='r')

# Get episode list
episodes = sorted([k for k in root.keys() if k.startswith('episode_')])
print(f"Found {len(episodes)} episodes")

# NPC Y offset (discovered in coordinate exploration - 8 bits!)
NPC_Y_OFFSET = -8.0


In [ ]:
list(episode.attrs['state_attributes'])

In [ ]:
# Load an episode
EPISODE_IDX = 0
episode = root[episodes[EPISODE_IDX]]

frames = episode['frames'][:]
state = episode['state'][:]
state_attrs = list(episode.attrs['state_attributes'])

print(f"Episode: {episodes[EPISODE_IDX]}")
print(f"Frames: {frames.shape}")
print(f"State: {state.shape}")
print(f"State attrs: {state_attrs}")


In [ ]:
# Extract positions and angles
def get_attr(name):
    return state[:, state_attrs.index(name)]

hero_x = get_attr('HeroGlobalPosX')
hero_y = get_attr('HeroGlobalPosY')
hero_z = get_attr('HeroGlobalPosZ')
hero_angle = get_attr('HeroAngle')

npc_x = get_attr('NpcGlobalPosX')
npc_y = get_attr('NpcGlobalPosY') + NPC_Y_OFFSET  # Apply 8-bit offset
npc_z = get_attr('NpcGlobalPosZ')
npc_angle = get_attr('NpcGlobalPosAngle')

print(f"Hero angle range: [{hero_angle.min():.2f}, {hero_angle.max():.2f}]")
print(f"NPC angle range: [{npc_angle.min():.2f}, {npc_angle.max():.2f}]")

# Check if angles are in radians or degrees
if abs(hero_angle.max()) > 2 * np.pi:
    print("Angles appear to be in DEGREES")
    ANGLE_UNIT = 'degrees'
else:
    print("Angles appear to be in RADIANS")
    ANGLE_UNIT = 'radians'


In [ ]:
# Plot static view of a single frame with angle cones
def plot_frame_with_angles(frame_idx, ax_map, ax_frame, cone_length=3.0, cone_angle=30):
    """Plot map view with angle cones and corresponding frame."""
    ax_map.clear()
    ax_frame.clear()
    
    # Get positions at this frame
    hx, hy = hero_x[frame_idx], hero_y[frame_idx]
    nx, ny = npc_x[frame_idx], npc_y[frame_idx]
    ha = hero_angle[frame_idx]
    na = npc_angle[frame_idx]
    
    # Convert to degrees if needed
    if ANGLE_UNIT == 'radians':
        ha_deg = np.degrees(ha)
        na_deg = np.degrees(na)
    else:
        ha_deg = ha
        na_deg = na
    
    # Plot trajectory up to this point
    ax_map.plot(hero_x[:frame_idx+1], hero_y[:frame_idx+1], 'b-', alpha=0.3, linewidth=1)
    ax_map.plot(npc_x[:frame_idx+1], npc_y[:frame_idx+1], 'r-', alpha=0.3, linewidth=1)
    
    # Plot current positions
    ax_map.scatter([hx], [hy], c='blue', s=100, zorder=5, label='Hero')
    ax_map.scatter([nx], [ny], c='red', s=100, zorder=5, label='NPC')
    
    # Draw angle cones (wedges)
    # Matplotlib Wedge: theta1 and theta2 are in degrees, CCW from +x axis
    
    # Hero cone (blue)
    hero_wedge = Wedge(
        (hx, hy), cone_length,
        ha_deg - cone_angle/2,
        ha_deg + cone_angle/2,
        alpha=0.4, color='blue'
    )
    ax_map.add_patch(hero_wedge)
    
    # NPC cone (red)
    npc_wedge = Wedge(
        (nx, ny), cone_length,
        na_deg - cone_angle/2,
        na_deg + cone_angle/2,
        alpha=0.4, color='red'
    )
    ax_map.add_patch(npc_wedge)
    
    # Draw direction arrows
    hero_dx = cone_length * np.cos(np.radians(ha_deg))
    hero_dy = cone_length * np.sin(np.radians(ha_deg))
    ax_map.arrow(hx, hy, hero_dx, hero_dy, head_width=0.3, head_length=0.2, fc='blue', ec='blue')
    
    npc_dx = cone_length * np.cos(np.radians(na_deg))
    npc_dy = cone_length * np.sin(np.radians(na_deg))
    ax_map.arrow(nx, ny, npc_dx, npc_dy, head_width=0.3, head_length=0.2, fc='red', ec='red')
    
    ax_map.set_aspect('equal')
    ax_map.legend(loc='upper right')
    ax_map.set_title(f'Frame {frame_idx} | Hero: {ha_deg:.1f}° | NPC: {na_deg:.1f}°')
    ax_map.set_xlabel('X')
    ax_map.set_ylabel('Y')
    ax_map.grid(True, alpha=0.3)
    
    # Set axis limits based on trajectory
    x_margin = (hero_x.max() - hero_x.min()) * 0.1
    y_margin = (hero_y.max() - hero_y.min()) * 0.1
    ax_map.set_xlim(hero_x.min() - x_margin, hero_x.max() + x_margin)
    ax_map.set_ylim(hero_y.min() - y_margin, hero_y.max() + y_margin)
    
    # Plot frame
    frame = frames[frame_idx]
    if frame.shape[0] == 3:  # CHW -> HWC
        frame = np.transpose(frame, (1, 2, 0))
    ax_frame.imshow(frame)
    ax_frame.set_title(f'Game Frame {frame_idx}')
    ax_frame.axis('off')

# Test on single frame
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
plot_frame_with_angles(100, ax1, ax2)
plt.tight_layout()
plt.show()


In [ ]:
# Create animation
fig, (ax_map, ax_frame) = plt.subplots(1, 2, figsize=(14, 6))

# Sample every N frames for smoother animation
SAMPLE_RATE = 3
frame_indices = list(range(0, len(frames), SAMPLE_RATE))
print(f"Animating {len(frame_indices)} frames (sampled every {SAMPLE_RATE})")

def animate(i):
    frame_idx = frame_indices[i]
    plot_frame_with_angles(frame_idx, ax_map, ax_frame, cone_length=4.0, cone_angle=45)
    return []

anim = FuncAnimation(fig, animate, frames=len(frame_indices), interval=100, blit=False)
plt.close()
HTML(anim.to_jshtml())


In [ ]:
# Check angle conventions - does the angle point towards the other entity?
print("Checking if angles point toward each other...")
print("="*60)

# Calculate the "correct" angle from hero to NPC
def angle_to_target(from_x, from_y, to_x, to_y):
    """Calculate angle from one point to another (in degrees)."""
    dx = to_x - from_x
    dy = to_y - from_y
    return np.degrees(np.arctan2(dy, dx))

# Sample a few frames
sample_frames = [50, 100, 200, 300, 400]
for fi in sample_frames:
    if fi >= len(hero_x):
        continue
    
    # Hero's recorded angle
    ha = hero_angle[fi]
    ha_deg = np.degrees(ha) if ANGLE_UNIT == 'radians' else ha
    
    # Calculated angle from hero to NPC
    calc_angle = angle_to_target(hero_x[fi], hero_y[fi], npc_x[fi], npc_y[fi])
    
    # Difference
    diff = (ha_deg - calc_angle + 180) % 360 - 180  # Normalize to [-180, 180]
    
    print(f"Frame {fi:3d}: Hero angle={ha_deg:7.1f}°, To NPC={calc_angle:7.1f}°, Diff={diff:+7.1f}°")

print("\n(If Diff is close to 0, hero is facing NPC)")
print("(If Diff is close to ±180, hero is facing away from NPC)")


In [ ]:
# Test different angle transformations to find the right convention
# Game engines often use different conventions:
# - Y-up vs Z-up
# - Clockwise vs counter-clockwise
# - Starting from +X vs +Y vs -Y

print("Testing angle transformations...")
print("="*70)

transformations = [
    ("Raw", lambda a: a),
    ("Negated", lambda a: -a),
    ("+90°", lambda a: a + 90),
    ("-90°", lambda a: a - 90),
    ("+180°", lambda a: a + 180),
    ("90° - angle", lambda a: 90 - a),
    ("-90° - angle", lambda a: -90 - a),
]

# Test each transformation on sample frames
sample_frames = [100, 200, 300]

for name, transform in transformations:
    total_diff = 0
    for fi in sample_frames:
        if fi >= len(hero_x):
            continue
        
        ha = hero_angle[fi]
        ha_deg = np.degrees(ha) if ANGLE_UNIT == 'radians' else ha
        transformed = transform(ha_deg)
        
        calc_angle = angle_to_target(hero_x[fi], hero_y[fi], npc_x[fi], npc_y[fi])
        diff = abs((transformed - calc_angle + 180) % 360 - 180)
        total_diff += diff
    
    avg_diff = total_diff / len(sample_frames)
    facing = "✓ FACING" if avg_diff < 45 else "✗ NOT FACING" if avg_diff > 135 else "~ SIDE"
    print(f"{name:15s}: avg_diff={avg_diff:6.1f}° {facing}")


In [ ]:
# Final visualization with corrected angles
# ADJUST THESE TRANSFORMS based on results from cell above!

HERO_ANGLE_TRANSFORM = lambda a: a  # e.g., lambda a: -a or lambda a: a + 90
NPC_ANGLE_TRANSFORM = lambda a: a   # Adjust if needed

def plot_frame_corrected(frame_idx, ax_map, ax_frame, cone_length=4.0, cone_angle=45):
    """Plot with corrected angle transforms and zoomed view."""
    ax_map.clear()
    ax_frame.clear()
    
    hx, hy = hero_x[frame_idx], hero_y[frame_idx]
    nx, ny = npc_x[frame_idx], npc_y[frame_idx]
    
    ha = hero_angle[frame_idx]
    na = npc_angle[frame_idx]
    
    ha_deg = np.degrees(ha) if ANGLE_UNIT == 'radians' else ha
    na_deg = np.degrees(na) if ANGLE_UNIT == 'radians' else na
    
    # Apply transforms
    ha_deg = HERO_ANGLE_TRANSFORM(ha_deg)
    na_deg = NPC_ANGLE_TRANSFORM(na_deg)
    
    # Plot trajectory
    ax_map.plot(hero_x[:frame_idx+1], hero_y[:frame_idx+1], 'b-', alpha=0.3, linewidth=1)
    ax_map.plot(npc_x[:frame_idx+1], npc_y[:frame_idx+1], 'r-', alpha=0.3, linewidth=1)
    
    # Plot positions
    ax_map.scatter([hx], [hy], c='blue', s=100, zorder=5, label='Hero')
    ax_map.scatter([nx], [ny], c='red', s=100, zorder=5, label='NPC')
    
    # Draw cones
    hero_wedge = Wedge((hx, hy), cone_length, ha_deg - cone_angle/2, ha_deg + cone_angle/2, alpha=0.4, color='blue')
    npc_wedge = Wedge((nx, ny), cone_length, na_deg - cone_angle/2, na_deg + cone_angle/2, alpha=0.4, color='red')
    ax_map.add_patch(hero_wedge)
    ax_map.add_patch(npc_wedge)
    
    # Direction arrows
    ax_map.arrow(hx, hy, cone_length*np.cos(np.radians(ha_deg)), cone_length*np.sin(np.radians(ha_deg)),
                 head_width=0.3, head_length=0.2, fc='blue', ec='darkblue', linewidth=2)
    ax_map.arrow(nx, ny, cone_length*np.cos(np.radians(na_deg)), cone_length*np.sin(np.radians(na_deg)),
                 head_width=0.3, head_length=0.2, fc='red', ec='darkred', linewidth=2)
    
    ax_map.set_aspect('equal')
    ax_map.legend(loc='upper right')
    ax_map.set_title(f'Frame {frame_idx} | Hero: {ha_deg:.1f}° | NPC: {na_deg:.1f}°')
    ax_map.set_xlabel('X')
    ax_map.set_ylabel('Y')
    ax_map.grid(True, alpha=0.3)
    
    # Auto-zoom to current position with padding
    padding = 15
    center_x = (hx + nx) / 2
    center_y = (hy + ny) / 2
    ax_map.set_xlim(center_x - padding, center_x + padding)
    ax_map.set_ylim(center_y - padding, center_y + padding)
    
    # Frame
    frame = frames[frame_idx]
    if frame.shape[0] == 3:
        frame = np.transpose(frame, (1, 2, 0))
    ax_frame.imshow(frame)
    ax_frame.set_title(f'Game Frame {frame_idx}')
    ax_frame.axis('off')

# Test
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
plot_frame_corrected(150, ax1, ax2)
plt.tight_layout()
plt.show()


In [ ]:
# Animation with corrected angles
fig, (ax_map, ax_frame) = plt.subplots(1, 2, figsize=(14, 6))

SAMPLE_RATE = 3
frame_indices = list(range(0, len(frames), SAMPLE_RATE))
print(f"Animating {len(frame_indices)} frames")

def animate_corrected(i):
    frame_idx = frame_indices[i]
    plot_frame_corrected(frame_idx, ax_map, ax_frame, cone_length=5.0, cone_angle=60)
    return []

anim = FuncAnimation(fig, animate_corrected, frames=len(frame_indices), interval=100, blit=False)
plt.close()
HTML(anim.to_jshtml())
